# 2.1 SOO for 2D scalar field

In [1]:
%matplotlib inline
import sys
import os

# Sort the paths out to run from this file
parent_path = os.path.dirname(os.getcwd())
src_path = os.path.join(os.path.sep,parent_path, 'src')
sys.path.append(src_path)
print(src_path)

/home/luke/code/cyclops/src


We load the stored true fields that we read in chapter 1 of the manual. We also load the graph manager to display the results.

We get the boundaries of the field, and make sure that sensors can't be placed at the very edge but must be placed at least 2mm away.

In [2]:
from run_me import *


# Load any objects necessary
pickle_manager = PickleManager()
graph_manager = PlotManager()
true_temp_field = pickle_manager.read_file('simulation', 'temp_plane_field.pickle')
grid = pickle_manager.read_file('simulation', 'temp_plane_points.pickle')

field_bounds = true_temp_field.get_bounds()
sensor_bounds = field_bounds+np.array([[1, 1], [-1, -1]])*0.002

ModuleNotFoundError: No module named 'run_me'

We use the symmetry manager to force any assumptions about planes of symmetry that we might make.

We then setup the sensor suite, and you can uncomment the code below to find out how to use a MultiSensor (that finds the value at many points - like a DIC or an IR camera).

In [ ]:
# Setup the symmetry
symmetry_manager = SymmetryManager()
symmetry_manager.set_2D_x(np.mean(field_bounds[:, 0]))

# Setup the sensor suite
def f(x): return np.zeros(x.shape)
sensor = RoundSensor(0, f, 0, np.array([[-5000], [5000]]), 0, 2)
sensors = np.array([sensor]*5)

# def f(x): return np.zeros(x.shape)
# sensor = MultiSensor(0, f, 0.1, np.array([[-5000], [5000]]), np.linspace(sensor_bounds[0, 0], sensor_bounds[1, 0], 10).reshape(-1, 2))
# sensors = np.array([sensor])

sensor_suite = SensorSuite(
    ScalarField(RBFModel, field_bounds), 
    sensors,
    symmetry=[symmetry_manager.reflect_2D_horiz]
)

We define the optimiser. You can change up the parameters including the time it takes to optimise for, and the type of optimiser (GAOptimiser is the only other SOO optimiser). Also look at the optional arguments you can use - for instance specifying how many cores you want the optimisation to run on in the plan_soo function.

In [ ]:
# Setup the experiment
optimiser = PSOOptimiser('00:00:10')
experiment = Experiment(
    true_temp_field,
    grid,
    optimiser
)
experiment.plan_soo(
    sensor_suite,
    sensor_bounds
)
res = experiment.design()
proposed_layout, true_temps, model_temps, sensor_values = experiment.get_SOO_plotting_arrays(res.X)

In [ ]:
# Display the results
graph_manager.build_optimisation(
    res.history
)
graph_manager.draw()
graph_manager.build_2D_compare(
    grid,
    proposed_layout,
    true_temps,
    model_temps
)
graph_manager.draw()
graph_manager.build_3D_compare(
    grid,
    true_temps,
    model_temps
)
graph_manager.draw()